In [15]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectPercentile, f_classif

import numpy as np
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [16]:
import pickle
with open('variaveis.pkl', 'rb') as f:
  X_train, Y_train, X_test, Y_teste = pickle.load(f)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [18]:
# Standardize the data
from sklearn.preprocessing import StandardScaler
X = X_train.values
X_std = StandardScaler().fit_transform(X)

X_t = X_test.values
X_t_std = StandardScaler().fit_transform(X_t)

In [19]:
# CRL + K + C -> COMENTAR VARIOS EM SIMULTANEO
# CTL + K + U -> DESCOMENTAR

In [20]:
from sklearn.decomposition import PCA

# Call the PCA method with 5 components. 
pca = PCA(n_components=5)
pca.fit(X_std)
X_pca = pca.transform(X_std)
X_t_pda = pca.transform(X_t_std)

In [21]:

# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               }

In [22]:
# Create the parameter grid based on the results of random search 


# Create the model
rdf_model = RandomForestClassifier(n_estimators=100, random_state = 0)

# Best model
opt_model_rf = GridSearchCV(rdf_model, random_grid,  scoring='accuracy', verbose=10)

# Fit the model
opt_model_rf.fit(X_pca, Y_train) 

print (opt_model_rf.best_estimator_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV 1/5; 1/108] START max_depth=10, min_samples_leaf=1, min_samples_split=2.....
[CV 1/5; 1/108] END max_depth=10, min_samples_leaf=1, min_samples_split=2;, score=0.993 total time=   0.1s
[CV 2/5; 1/108] START max_depth=10, min_samples_leaf=1, min_samples_split=2.....
[CV 2/5; 1/108] END max_depth=10, min_samples_leaf=1, min_samples_split=2;, score=0.993 total time=   0.1s
[CV 3/5; 1/108] START max_depth=10, min_samples_leaf=1, min_samples_split=2.....
[CV 3/5; 1/108] END max_depth=10, min_samples_leaf=1, min_samples_split=2;, score=0.970 total time=   0.1s
[CV 4/5; 1/108] START max_depth=10, min_samples_leaf=1, min_samples_split=2.....
[CV 4/5; 1/108] END max_depth=10, min_samples_leaf=1, min_samples_split=2;, score=0.970 total time=   0.1s
[CV 5/5; 1/108] START max_depth=10, min_samples_leaf=1, min_samples_split=2.....
[CV 5/5; 1/108] END max_depth=10, min_samples_leaf=1, min_samples_split=2;, score=0.948 total time=   0.

In [28]:
arvore = RandomForestClassifier(max_depth=10, random_state = 0)
arvore.fit(X_pca, Y_train)
arvore

RandomForestClassifier(max_depth=10, random_state=0)

In [29]:
previsoes = arvore.predict(X_t_pda)
confusion = pd.crosstab(Y_teste,previsoes, rownames=['Actual'], colnames=['Predicted'], margins=True)
print(confusion )

Predicted   0   1   2   3  All
Actual                        
0          43   1   0   0   44
1           0  38   2   0   40
2           1   1  38   0   40
3           0   0   1  44   45
All        44  40  41  44  169


In [30]:
# Accuracy
score1 = accuracy_score(Y_teste,previsoes)
#score2= accuracy_score(Y_train,predictions1)

print('Accuracy:{0:f}'.format(score1))

Accuracy:0.964497


In [31]:
# Report
print(classification_report(Y_teste,previsoes))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98        44
           1       0.95      0.95      0.95        40
           2       0.93      0.95      0.94        40
           3       1.00      0.98      0.99        45

    accuracy                           0.96       169
   macro avg       0.96      0.96      0.96       169
weighted avg       0.96      0.96      0.96       169



In [32]:
# Validate model using best model and cross validation
pecc_dt = cross_val_score(rdf_model,  X_pca, Y_train, cv = 10).mean()  # ...,X_pca,...
pecc_dt

0.9688103599648814